# Lab02: Frequent itemset mining

- Student ID: 20120547
- Student name: Võ Thành Phong

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret=list(set(a) | set(b))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support=0
        for i in self.data.keys():
            if len(set(itemset)) == len(set(itemset).intersection(self.data[i])):
                support+=1
        return support/len(list(self.data.keys()))

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        if k==1:
            minSup=self.minSup
        elif k>1:
            minSup=self.minSup/len(list(self.data.keys()))
        for i in search_space.keys():
            if search_space[i]['support']>=minSup:
                self.L[k].append(search_space[i]['itemset'])
            else:
                search_space[i]['pruned']=True
        if self.L[k]==[]:
            del self.L[k]


    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset=joinset(search_space[a]['itemset'],search_space[b]['itemset'])
                sp_newset=self.computeItemsetSupport(newset)
                # Second add newset to search_space
                search_space[a][b]={'itemset':list(newset),'pruned':False,'support':sp_newset}
                sub_search_space[b]=search_space[a][b]
                sub_tree[b]={}
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')

In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]], 3: [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [40

### Answer questions here:
**Why don't we compute support of items while reading data?**
- Bởi vì, thứ nhất là do các bước đọc dữ liệu từ hàm readData không thể biết trước được số lượng transactions có trong dữ liệu. Bộ dữ liệu được đọc bằng cách duyệt qua lần lượt từng transaction để đọc tất cả item vào đối tượng s, nếu duyệt qua toàn bộ để biết được số lượng transaction sẽ lập code làm tăng thời gian không đáng có.
- Hơn nữa việc tính support cho itemset sẽ được lặp lại nhiều lần về sau cho các L-itemsets khác (với L>1). Do đó viết một hàm tính support trong class TP một cách phù hợp sẽ hữu ích cho các bước sau.

**why should we do sort**
- Việc sắp xếp các item trong self.s theo thứ tự tăng dần của support count sẽ giúp cho việc nếu xác định được một itemset là không phổ biến thì nó sẽ có số lượng item là nhỏ nhất.
- Ví dụ ta có các cặp item:support count trong self.s được xếp lộn xộn như sau: {3:3, 1:1, 2:3} với min support count là 2 thì theo thuật toán Tree Projection được cài đặt bên trên, itemset [3] thỏa là tập phổ biến nên sẽ tiếp tục tạo không gian tìm kiếm cho trường hợp k=2 bắt đầu từ 3 và [3,1] và [3,2] sẽ được thêm vào search space rồi mới thực hiện tỉa. Trong khi có thể bỏ qua tất cả lần duyệt các itemset có chứa item 1 nếu xét item 1 đầu tiên ngay từ đầu.

**study about python set and its advantages ?**
- Set trong Python là kiểu dữ liệu tập hợp không có thứ tự, có thể thay đổi và không có phần tử trùng lặp. Set được ký hiệu bởi cặp dấu ngoặc nhọn {}. Các phần tử của set sẽ được đặt trong cặp ngoặc nhọn.
- Set không có tính thứ tự do việc cài đặt set là bằng bảng băm, chúng ta sẽ không biết thứ tự lưu các phần tử vào set như thế nào mà tùy thuộc vào cách băm. Do việc cài đặt bàng bảng băm nên set có các ưu điểm chính như sau:
  + Điều này cho phép các phép toán trên set được thực hiện với độ phức tạp thời gian trung bình là O(1). Khi kiểm tra xem một phần tử có trong set hay không Python chỉ cần tính toán mã băm của phần tử và kiểm tra xem phần tử có trong bảng băm hay không.
  + Set không chứa các phần tử trùng lặp, nên rất dễ dàng để thực hiện phép hội.
  + Set hỗ trợ mạnh về các phép toán liên quan đến tập hợp như giao, hợp, hiệu, .... Do đó việc kiểm tra phần tử có trong set trở nên rất dễ dàng.
- Vậy lợi ích của việc dùng set trong khi cài đặt thuật toán Tree Projection:
  + Thực hiện việc hợp(join) 2 itemset trong hàm joinset sẽ loại bỏ được việc trùng lặp phần tử ngay lập tức mà không cần code quá phức tạp.
  + Khi tính support cho các L-itemsets (L>1) trong hàm computeItemsetSupport, ta kiểm tra các item của một itemset có cùng xuất hiện trong một transaction hay không rất dễ dàng qua phép toán giao (intersection) trên set.
  + Khi bộ dữ liệu rất lớn thì làm việc trên set giúp tăng tốc độ hơn rất nhiều so với trên list.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- Việc dùng pruned = True/False để cho biết itemset có là phổ biến hay không mà không xóa trực tiếp trong search_space và tree là do thuật toán được cài đặt tìm kiếm các itemset phổ biến một cách đệ qui bắt đầu từ không gian tìm kiếm cho k=1 và tăng dần lên k=2, 3, 4, ....
- Và ở không gian tìm kiếm cho k+1 thì tree và search_space sẽ lại là sub_tree và sub_searchspace với các bộ key-value hoàn toàn mới được kế thừa từ việc kết hợp các itemset ở không gian k. Do đó nếu có xóa trực tiếp thì cuối cùng nếu tất cả 1-itemset ban đầu đều là tập phổ biến thì search_space và tree vẫn chứa đầy đủ các nhanh và lúc này không có cách để nhận biết đâu là tập không phổ biến và nhánh nên bị tỉa.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

**1. Về cấu trúc dữ liệu:**
  - Apriori dùng cấu trúc dạng tập hợp (như list, bảng) để lưu các tập ứng viên C và các tập phổ biến L.
  - Tree Projection sẽ lưu các itemset trong các node của cấu trúc cây.
**2. Cách tìm tập phổ biến:**
  - Apriori sẽ tạo tập ${C}_{k+1}$ bằng cách kết hợp tất cả các itemset trong ${L}_{k}$ lại với nhau. Rồi sau đó duyệt lần lượt qua hết ${C}_{k+1}$ để tìm những itemset thỏa min support và thêm vào ${L}_{k+1}$.
  - Tree Projection sẽ lưu ở mức 1 của cây các 1-itemset ứng viên và sau đó loại ngay lập tức các itemset ứng viên không là tập phổ biến, sau đó từ các 1-itemset phổ biến còn lại tạo tiếp **theo chiều sâu** các L-itemset ứng viên (L>1) ở các mức cao hơn của cây, và vẫn tiến hành loại ngay những itemset nào không phổ biến.
  - Ví dụ ta có các item ban đầu của database là 1, 2, 3, 4. **Giả sử:** itemset [1] sẽ không thỏa min support, các itemset [2], [3], [4] và tất cả các itemset khác được tạo ra sau này đều sẽ thỏa min support, ta có hình ảnh như sau:
  ![img](https://res.cloudinary.com/vtphong/image/upload/v1681221934/data-mining/image1.png)
  - Đầu tiên là xóa itemset [1], sau đó tạo các itemset khác theo chiều các mũi tên màu đỏ và cuối cùng mới đến tạo itemset [3, 4].
**3. Về hiệu suất:**
  - Thuật toán Tree Projection thường cho hiệu suất tốt hơn về mặt tốc độ so với thuật toán Apriori, đặc biệt là khi xử lý các tập dữ liệu lớn và các tập phổ biến có kích thước lớn.

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

**Trước tiên chuyển file 'churn.txt' sang flat file với định dạng là txt bằng hàm convertDataset tự định nghĩa sau đây.**

**Mô tả ý tưởng tạo data:**
- Dòng đầu trong file churn chứa tên thuộc tính, các dòng sau chứa từng giá trị cho mỗi thuộc tính của mỗi khách hàng.
- Do đó ta có thể tạo data có dạng như sau:
  + Ví dụ mỗi hàng của file chứa data đã chuyển đổi sẽ tương ứng là thông tin của mỗi khách hàng trong file churn.txt. Khi đó file data đã chuyển đổi sẽ có dạng:
  
<thuộc tính 1>:<giá trị 1_1>;<thuộc tính 2>:<giá trị 1_2>;...;<thuộc tính n>:<giá trị 1_n>
  
<thuộc tính 1>:<giá trị 2_1>;<thuộc tính 2>:<giá trị 2_2>;...;<thuộc tính n>:<giá trị 2_n>
  
...
  
<thuộc tính 1>:<giá trị m_1>;<thuộc tính 2>;<giá trị m_2>;...;<thuộc tính n>:<giá trị m_n>
  
  + **Với n là số thuộc tính ban đầu trong file churn.txt, m là số khách hàng ban đầu trong file churn.txt**.
  + **Đặt tên là 'data.txt'**.

In [6]:
def convertDataset(path):
    columns_name=["State","Account Length","Area Code","Phone","Int'l Plan","VMail Plan","VMail Message",
                  "Day Mins","Day Calls","Day Charge","Eve Mins","Eve Calls","Eve Charge","Night Mins",
                  "Night Calls","Night Charge","Intl Mins","Intl Calls","Intl Charge","CustServ Calls","Churn?"]
    f = open(path, 'r')
    writer = open('data.txt','w')
    temp=f.readline() #loại bỏ dòng đầu là các thuộc tính
    for line in f:
        data = line.split(',')
        data[-1]=data[-1].replace('.','')
        for i in range(len(columns_name)):
            data[i]=columns_name[i]+':'+data[i]
        row=';'.join(data)
        writer.write(row)
    f.close()
    writer.close()

In [7]:
convertDataset('churn.txt')

**Mô tả ý tưởng phân tích dữ liệu churn:**
- Churn là dữ liệu khách hàng rời bỏ dịch vụ. Với các thông tin liên quan thì khách hàng có rời bỏ dịch vụ hay không?
- Vậy ta có thể nghĩ đến việc tìm các tập phổ biến có chứa yếu tố churn, sau đó tím ra các luật kết hợp mà yếu tố churn nằm ở vế phải thỏa một min confidence nào đó, tất nhiên những yếu tố ở vế trái của luật sẽ là những yếu tố ảnh hưởng đến việc khách hàng có rời bỏ dịch vụ hay không.

**Tận dụng lại hàm readData, và lớp TP để tìm tập phổ biến.** Nhưng cần định nghĩa lại một chút hàm readDataset để có thể lưu các itemset kiểu string.

In [8]:
def readDataset(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            line=line.strip()
            itemset=set(line.split(';')) 
            for item in itemset:  
                s[item]+=1    
            data[tid]= itemset
            tid+=1
    
    return data, s

In [9]:
class ChurnAnalysis:
    def __init__(self, data=None, s=None, L=None, attribute=None, minConf=None):
        self.data=data
        self.s=s
        self.L=L
        self.attribute=attribute
        self.minConf=minConf
        self.filtered_L={}
        self.AscRules=[]
        
    def joinset(a, b):
        ret=list(set(a) | set(b))
        return ret
    
    def filterFrequentItemsets(self):
        L2={}
        for k in self.L.keys():
            if k==1:
                continue
            L2[k]=[]
            for i in range(len(self.L[k])):
                L2[k].append([])
                check_exist=False
                for item in self.L[k][i]:
                    if self.attribute in item:
                        check_exist=True
                if check_exist==True:
                    for item in self.L[k][i]:
                        L2[k][i].append(item)
            for i in L2[k]:
                if len(i)==0:
                    L2[k].remove(i)
            if len(L2[k])==0:
                del L2[k]
        return L2
    
    def computeItemsetSupport(self,itemset):
        support=0
        for i in self.data.keys():
            if len(set(itemset)) == len(set(itemset).intersection(self.data[i])):
                support+=1
        return support/len(self.data.keys())
    
    def computeRuleSupport(self, leftSide, rightSide):
        return self.computeItemsetSupport(joinset(leftSide, rightSide))
    
    def computeRuleConfidence(self, leftSide, rightSide):
        return self.computeItemsetSupport(joinset(leftSide, rightSide))/self.computeItemsetSupport(leftSide)
    
    def generateARule(self,itemset):
        itemset_copy=itemset.copy()
        leftSide, rightSide=[], []
        rule={}
        for i in itemset:
            if self.attribute in i:
                rightSide.append(i)
                itemset_copy.remove(i)
                leftSide=itemset_copy
                break
        rule[tuple(leftSide)]={}
        rule[tuple(leftSide)]['result']=rightSide
        rule[tuple(leftSide)]['support']=self.computeRuleSupport(leftSide, rightSide)
        rule[tuple(leftSide)]['confidence']=self.computeRuleConfidence(leftSide, rightSide)
        return rule
    
    def generateAllRules(self):
        self.filtered_L=self.filterFrequentItemsets()
        for k in self.filtered_L.keys():
            for i in self.filtered_L[k]:
                rule=self.generateARule(i)
                if list(rule.values())[0]['confidence']>=self.minConf:
                    self.AscRules.append(rule)
        return self.AscRules

In [10]:
data2, s2=readDataset('data.txt')

In [11]:
minSup=int(0.5*len(list(data2.keys()))) #minSup bằng 50% số lượng transactions
frequentItemsets=TP(data=data2,s=s2, minSup=minSup)

In [12]:
rules=ChurnAnalysis(data=data2,s=s2,L=frequentItemsets.miningResults(),attribute='Churn?',minConf=0.0)

In [13]:
rules.generateAllRules()

[{('VMail Plan:no',): {'result': ['Churn?:False'],
   'support': 0.6024602460246025,
   'confidence': 0.8328494400663625}},
 {('VMail Message:0',): {'result': ['Churn?:False'],
   'support': 0.6024602460246025,
   'confidence': 0.8328494400663625}},
 {("Int'l Plan:no",): {'result': ['Churn?:False'],
   'support': 0.7992799279927992,
   'confidence': 0.8850498338870432}},
 {('VMail Plan:no', 'VMail Message:0'): {'result': ['Churn?:False'],
   'support': 0.6024602460246025,
   'confidence': 0.8328494400663625}},
 {('VMail Plan:no', "Int'l Plan:no"): {'result': ['Churn?:False'],
   'support': 0.5634563456345635,
   'confidence': 0.861467889908257}},
 {("Int'l Plan:no", 'VMail Message:0'): {'result': ['Churn?:False'],
   'support': 0.5634563456345635,
   'confidence': 0.861467889908257}},
 {('VMail Plan:no',
   "Int'l Plan:no",
   'VMail Message:0'): {'result': ['Churn?:False'], 'support': 0.5634563456345635, 'confidence': 0.861467889908257}}]

- Em tạo một lớp tên là ChurnAnalysis để phân tích tập churn như sau:
  + Phương thức filterFrequentItemsets sẽ lọc ra các tập L-itemsets (L>=2) là tập hợp những tập phổ biến chứa yếu tố churn nhưng có từ 2 phần tử trở lên (do để tạo luật từ itemset phải có từ 2 phần tử trở lên trong itemset).
  + Phương thức computeRuleConfidence tính confidence cho luật.
  + Phương thức generateAllRules dùng để sinh tất cả các luật mà vế phải là yếu tố churn thỏa min confidence.

- Để không mất tính tổng quát cho lớp và các phương thức, có thể đổi yếu tố churn thành yếu tố khác và min confidence là một giá trị khác để truyền vào lớp và sinh các luật liên quan.
- Có thể thấy list trên chứa các dictionary với mỗi dictionary là một luật:
  + Key sẽ là vế trái của luật.
  + 'result' sẽ là vế phải của luật.
  + support là support của luật.
  + confidence là confidence của luật.

- Có thể thấy trong dataset này với min support bằng 50% thì không có tập phổ biến nào chứa churn=True (khách hàng rời bỏ dịch vụ).
- Có nhiều tập phổ biến chứa churn=False, vậy ta sẽ phân tích theo hướng kết hợp những yếu tố nào để tăng khả năng giữ lại khách hàng. Và để làm được điều này thì những luật ở trên là sự gợi ý tốt để tham khảo.
- **Để xem tất cả các tập phổ biến khi chưa lọc theo thuộc tính được chọn, có thể dùng thuộc tính .L**.
- **Để xem tất cả các tập phổ biến khi đã lọc theo thuộc tính được chọn, có thể dùng thuộc tính .filtered_L**.

In [14]:
print('Tất cả các tập phổ biến với min support đã cho:\n')
rules.L

Tất cả các tập phổ biến với min support đã cho:



{1: [['VMail Plan:no'],
  ['VMail Message:0'],
  ['Churn?:False'],
  ["Int'l Plan:no"]],
 2: [['VMail Plan:no', 'VMail Message:0'],
  ['VMail Plan:no', 'Churn?:False'],
  ['VMail Plan:no', "Int'l Plan:no"],
  ['Churn?:False', 'VMail Message:0'],
  ["Int'l Plan:no", 'VMail Message:0'],
  ['Churn?:False', "Int'l Plan:no"]],
 3: [['VMail Plan:no', 'Churn?:False', 'VMail Message:0'],
  ['VMail Plan:no', "Int'l Plan:no", 'VMail Message:0'],
  ['VMail Plan:no', 'Churn?:False', "Int'l Plan:no"],
  ['Churn?:False', "Int'l Plan:no", 'VMail Message:0']],
 4: [['Churn?:False', 'VMail Plan:no', "Int'l Plan:no", 'VMail Message:0']]}

**Nếu trên phương diện là chủ dịch vụ, để chắc chắn là không có yếu tố ảnh hưởng nào làm cho churn=true đáng lo ngại xảy ra, em sẽ xem xét tập dữ liệu với một min support rất thấp hơn (20% cũng đã đủ làm em lo lắng 😁)**

In [15]:
minSup2=int(0.2*len(list(data2.keys()))) #minSup bằng 20% số lượng transactions
frequentItemsets2=TP(data=data2,s=s2, minSup=minSup2)

In [16]:
rules2=ChurnAnalysis(data=data2,s=s2,L=frequentItemsets2.miningResults(),attribute='Churn?',minConf=0.0)

In [17]:
rules2.generateAllRules()

[{('CustServ Calls:2',): {'result': ['Churn?:False'],
   'support': 0.20162016201620162,
   'confidence': 0.8853754940711462}},
 {('Area Code:408',): {'result': ['Churn?:False'],
   'support': 0.2148214821482148,
   'confidence': 0.8544152744630071}},
 {('Area Code:510',): {'result': ['Churn?:False'],
   'support': 0.2145214521452145,
   'confidence': 0.8511904761904762}},
 {('VMail Plan:yes',): {'result': ['Churn?:False'],
   'support': 0.2526252625262526,
   'confidence': 0.9132321041214749}},
 {('CustServ Calls:1',): {'result': ['Churn?:False'],
   'support': 0.3177317731773177,
   'confidence': 0.8966977138018628}},
 {('Area Code:415',): {'result': ['Churn?:False'],
   'support': 0.42574257425742573,
   'confidence': 0.8574018126888218}},
 {(): {'result': [], 'support': 1.0, 'confidence': 1.0}},
 {('VMail Plan:no',): {'result': ['Churn?:False'],
   'support': 0.6024602460246025,
   'confidence': 0.8328494400663625}},
 {(): {'result': [], 'support': 1.0, 'confidence': 1.0}},
 {('VMa

**Vẫn không có yếu tố nào làm cho churn = True xảy ra mà làm cho yếu tố này thành phổ biến.**

# 4 References

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
